Họ và tên: Võ Hồng Quân


MSSV: 22134012


Lớp: 22134

In [72]:
import numpy as np
import scipy.io

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import sklearn
from sklearn.model_selection import train_test_split

import plotly.graph_objects as go 
from torch.utils.tensorboard import SummaryWriter

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [73]:
writer = SummaryWriter()

In [64]:
class SplitDataset:
    def __init__(self, path= '', train_test_split_size= []):
        self.data = scipy.io.loadmat(path)
        self.inputs = self.data['X']
        self.targets = np.ravel(self.data['y'])
        self.train_test_split_size = train_test_split_size

        # Modify label 1-10 into 0-9
        self.targets[self.targets == 10] = 0

    def __call__(self):
        if self.train_test_split_size != []:
            train_size, test_size, valid_size = self.train_test_split_size
            # print(train_size, test_size, valid_size)

            self.X_new, self.X_test, self.y_new, self.y_test = train_test_split(self.inputs, self.targets, train_size= train_size, test_size= test_size)
            self.X_train, self.X_valid, self.y_train, self.y_valid = train_test_split(self.X_new, self.y_new, test_size= valid_size)

        self.train = [self.X_train, self.y_train]
        self.test = [self.X_test, self.y_test]
        self.valid = [self.X_valid, self.y_valid]
        
        return self.train, self.test, self.valid


In [65]:
class GetDataset(Dataset):
    def __init__(self, transforms= None, dataset= None):
        super().__init__()
        self.transform = transforms
        self.inputs, self.targets = dataset
        

    def __getitem__(self, name):
        self.sample = self.inputs, self.targets

        if self.transform:
            self.sample = self.transform(self.sample)

        return self.sample
    
    def __len__(self):
        return len(self.inputs)

In [66]:
class Tranform: 
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [76]:
train_array, test_array, valid_array = SplitDataset(path= 'ex7data.mat', train_test_split_size= [0.7, 0.2, 0.1]).__call__()

In [77]:
train_tensor = GetDataset(transforms= Tranform(), dataset= train_array)
test_tensor = GetDataset(transforms= Tranform(), dataset= test_array)
valid_tensor = GetDataset(transforms= Tranform(), dataset= valid_array)

In [82]:
# Visualize images
train_images, train_labels = train_tensor[0]
train_images = train_images.reshape(-1, 1, 20 ,20)
grid = torchvision.utils.make_grid(train_images)

writer.add_image('Train images', grid)

In [71]:
train[0].shape

(3150, 400)

In [80]:
next(iter(train_tensor))

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64),
 tensor([7, 6, 3,  ..., 2, 2, 9], dtype=torch.uint8))

In [81]:
train_tensor[0]

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64),
 tensor([7, 6, 3,  ..., 2, 2, 9], dtype=torch.uint8))